In [1]:
manufacturer = 'GMB'
type_code = '2208'

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()
list_part_number = df['Part_Number'].tolist()

print('总数量：' + str(len(list_url)))
list_url

总数量：1081


['https://www.rockauto.com/en/parts/gmb,1101010,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101030,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101040,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101040AL,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101040P,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101050,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101060,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101070,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101080,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101080P,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101090,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1101090P,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1102052,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1102908,water+pump,2208',
 'https://www.rockauto.com/en/parts/gmb,1103075,water+pump,2208',
 'htt

In [4]:
df = pd.DataFrame()

df

""


In [5]:
import requests

In [6]:
i = 0

url = list_url[i]
part_number = list_part_number[i]

resp = requests.get(url).text

resp

'<!DOCTYPE HTML>\n<html class="no-js " lang="en">\n<head>\n<meta http-equiv="content-type" content="text/html; charset=windows-1252">\n<meta name="description" content="RockAuto ships auto parts and body parts from over 300 manufacturers to customers\' doors worldwide, all at warehouse prices. Easy to use parts catalog.">\n<meta name="keywords" content=" Water Pump,,,, Water Pump, GMB,">\n<meta name="Classification" content="automotive">\n<meta name="abstract" content="Online Store for brand name auto parts at warehouse-direct prices...">\n<meta name="robots" content="all">\n<meta http-equiv="Content-Security-Policy" content="script-src \'self\' \'unsafe-inline\' \'unsafe-eval\' *.rockauto.com; base-uri \'self\' *.rockauto.com; ">\n\n<link rel="canonical" href="https://www.rockauto.com/en/parts/gmb,1101010,water+pump,2208" />\n<link rel="alternate" href="https://www.rockauto.com/en/parts/gmb,1101010,water+pump,2208" hreflang="en" />\n<link rel="alternate" href="https://www.rockauto.com

In [7]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(resp, 'lxml')

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE HTML>
<html class="no-js" lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="RockAuto ships auto parts and body parts from over 300 manufacturers to customers' doors worldwide, all at warehouse prices. Easy to use parts catalog." name="description"/>
  <meta content=" Water Pump,,,, Water Pump, GMB," name="keywords"/>
  <meta content="automotive" name="Classification"/>
  <meta content="Online Store for brand name auto parts at warehouse-direct prices..." name="abstract"/>
  <meta content="all" name="robots"/>
  <meta content="script-src 'self' 'unsafe-inline' 'unsafe-eval' *.rockauto.com; base-uri 'self' *.rockauto.com; " http-equiv="Content-Security-Policy"/>
  <link href="https://www.rockauto.com/en/parts/gmb,1101010,water+pump,2208" rel="canonical"/>
  <link href="https://www.rockauto.com/en/parts/gmb,1101010,water+pump,2208" hreflang="en" rel="alternate"/>
  <link href="https://www.rockauto.com/es/parts/gmb,1101010,

In [9]:
from lxml import etree

In [10]:
html = etree.HTML(str(soup))

list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

list_part

[<Element tbody at 0x12c8a0280>]

In [11]:
list_part_number = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]')[0].text for _ in list_part]

list_part_number

['1101010']

In [12]:
import numpy as np

In [13]:
number_index = np.where(np.array(list_part_number) == part_number)[0]

number_index

array([0])

In [14]:
list_manufacturer = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]')[0].text for _ in list_part]

list_manufacturer

['GMB']

In [15]:
manufacturer_index = np.where(np.array(list_manufacturer) == manufacturer)[0]

manufacturer_index

array([0])

In [16]:
index = list(np.intersect1d(number_index, manufacturer_index))

index

[0]

In [17]:
index = index[0]

index

0

In [18]:
list_span = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]') for _ in list_part]
for _ in list_span:
    note_1 = _[0].xpath('./span[contains(@class, "span-link-underline-remover")]/text()')
    if len(note_1) == 0:
        note_1 = ''
    else:
        note_1 = note_1[0].strip()
        
note_1

'OE Replacement'

In [19]:
list_span = [_.xpath('./descendant::div[contains(@class, "listing-text-row") and not(contains(@class, "moreinfo-truck"))]') for _ in list_part]
for _ in list_span:
    note_2 = _[0].xpath('./span/span/text()')
    if len(note_2) == 0:
        note_2 = ''
    else:
        note_2 = [_.strip() for _ in note_2]
        while '' in note_2:
            note_2.remove('')
        note_2 = '\n'.join(note_2)
        
note_2

'Category: Water Pump'

In [20]:
list_span = [_.xpath('./descendant::span[contains(@title, "Replaces these Alternate/ OE Part Numbers")]') for _ in list_part]
for _ in list_span:
    if len(_) == 0:
        oe = ''
    else:
        oe = _[0].xpath('./text()')[0].replace(', ', ';')
        
oe

'8125501;8126772;8129459;J8125501;J8126772;J8129459;J9990266;J9990757;JR775048'

In [21]:
list_span = [_.xpath('./descendant::img[contains(@id, "inlineimg_thumb")]') for _ in list_part]
for _ in list_span:
    if len(_) == 0:
        src = ''
    else:
        src = 'https://www.rockauto.com' + _[0].xpath('./@src')[0].replace('__ra_m.', '.')

src

'https://www.rockauto.com/info/424/110-1010_ANG.jpg'

In [22]:
import json

In [23]:
list_span = [_.xpath('./descendant::input[contains(@id, "listing_data_essential")]') for _ in list_part]
for _ in list_span:
    if len(_) == 0:
        part_key = ''
    else:
        part_key = json.loads(_[0].xpath('./@value')[0])['partkey']

part_key

'7584148'

In [24]:
df_temp = pd.DataFrame([{'status': 'ok',
                         'Manufacturer': manufacturer,
                         'Part_Number': part_number,
                         'Vehcile': '',
                         'Alternate_OE_Part_Numbers': oe,
                         'Note_1': note_1,
                         'Note_2': note_2,
                         'Pic': '',
                         'Url': url,
                         'Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + type_code + '&pk=' + part_key,
                         'Src': src,
                         'Type_Code': type_code,
                         'Part_Key': part_key,}])

df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df

,status,Manufacturer,Part_Number,Vehcile,Alternate_OE_Part_Numbers,Note_1,Note_2,Pic,Url,Info,Src,Type_Code,Part_Key
0,ok,GMB,1101010,,8125501;8126772;8129459;J8125501;J8126772;J812...,OE Replacement,Category: Water Pump,,"https://www.rockauto.com/en/parts/gmb,1101010,...",https://www.rockauto.com/en/moreinfo.php?pt=22...,https://www.rockauto.com/info/424/110-1010_ANG...,2208,7584148
